<a href="https://www.kaggle.com/code/abioduntiamiyu/ashbourne-2022-trial-balance-ledger-analysis?scriptVersionId=244978030" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 10 12:54:49 2025

@author: Abiodun Tiamiyu

Business Task: Materiality Assessment for Year-End General Ledger

Business Question:
    We are conducting a year-end financial audit for Ashbourne Industries Inc.
    and require your assistance in performing a materiality assessment on the
    general ledger accounts.

Scope of Work:
    Use the December (year-end) balances from the provided general ledger Excel
    file. Monthly data (January-November) can be ignored. Determine which accounts 
    are material, based on the materiality threshold.
    
Business Instructions:
    1.  Summarise account balances in the summary table using the following code
        groupings.
                10000 series: Assets
                20000 series: Liabilities and Equity
                30000 series: Revenue
                40000-80000 series: Expenses
    2.  Calculate pre-tax income:
            Net income = Revenue - Expenses
            Taxes = Sum of accounts 81000-82000
            Pre-tax income = Net income + Taxes
    3.  Materiality Threshold:
            Overall materiality = 1% of pre-tax income
            Performance materiality = 75% of the above amount
    4.  Materiality Check:
            Identify and flag all accounts whose December balances exceed the 
        performance materiality threshold.
"""
import pandas as pd
    
    # Load the Excel file
file_path = "Ashbourne 2022 Trial Balance.xlsx"
df = pd.read_excel(file_path)

    # Step 1: Changing "Unnamed: 0" variable to "Code"
    # Then filter for required columns ("Code", "Account Name", "DEC")
df.info()
df.rename(columns = {"Unnamed: 0": "Code"}, inplace = True)
df.info()
filtered_df = df[["Code", "Account Name", "DEC"]]

#%% 
   # Step 2: Categorize accounts based on Code ranges.
def get_category(Code):
    if 10000 <= Code < 20000:
        return "Assets"
    elif 20000 <= Code < 30000:
        return "Liabilities and Equity"
    elif 30000 <= Code < 40000:
        return "Revenue"
    elif 40000 <= Code < 90000:
        return "Expenses"
    else:
        return "Other"
filtered_df["Category"] = filtered_df["Code"].apply(get_category)

#%%
    # Step 3: Calculate Summary totals.
summary_df = filtered_df.groupby("Category")["DEC"].sum()
revenue = summary_df.get("Revenue", 0)
expenses = summary_df.get("Expenses", 0)

#%%
    # Step 4: Calculate taxes from under 81000 and 82000
taxes = filtered_df[df["Code"].isin([81000, 82000])]["DEC"].sum()

    # Step 5 = Calculate pre-tax income, materiality, performance materiality
net_income = revenue - expenses
pre_tax_income = net_income + taxes
materiality = pre_tax_income * 0.01
performance_materiality = materiality * 0.75

#%%
    # Step 6: Flag material accounts
filtered_df.loc[df["DEC"].abs() > performance_materiality, "Is Material"] = True

    # Save Output to new Excel
output_path = "Ashbourne_Materiality_Flagged.xlsx"
filtered_df.to_excel(output_path, index = False)
output_path2 = "Summary_Totals_of_Code.xlsx"
summary_df.to_excel(output_path2, index = True)